# **Лабораторная работа 2**

В этой лабораторной работе необхадимо сделать свой реализацию алгоритмов машинного обучения, и сравнить её с реализацией на sklearn для 2 датасетов, полученных в предыдущей лабораторной работе.

In [2]:
import pandas
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import check_random_state
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import time
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
import sys

Заготавливаем функцию, которая будет отвечать за обучение, тестирование, подсчёт точности и др.

In [495]:
def DisplayMetrics(Method, X, Y, folds = 5, average = 'macro'):
  kf = StratifiedKFold(n_splits = folds, random_state = 128, shuffle = True)
  precision = np.zeros(folds)   
  recall = np.zeros(folds)  
  testAc = np.zeros(folds)
  trainAc = np.zeros(folds)
  X=X.astype(np.float32)
  for step, (trainI, valI) in enumerate(kf.split(X, Y)):
    TrX, TrY = X.loc[trainI].to_numpy(), Y.loc[trainI].to_numpy()
    ValX, ValY = X.loc[valI].to_numpy(), Y.loc[valI].to_numpy()
    Method.fit(TrX, TrY)
    PredY = Method.predict(ValX)
    PredTrY = Method.predict(TrX)
    precision[step] = precision_score(ValY, PredY, average = average)
    recall[step] = recall_score(ValY, PredY, average = average)
    trainAc[step] = accuracy_score(TrY, PredTrY)
    testAc[step] = accuracy_score(ValY, PredY)
  print("precision:", precision.mean())
  print("recall:", recall.mean())
  print("train_accuracy:", trainAc.mean())
  print("test_accuracy:", testAc.mean())

## **Реализация своими руками**

Логистическая регрессия

In [72]:
class LogReg():
    def __init__(self, learning_rate = 0.01, grad_iters=100):
        self.lr = learning_rate
        self.gi = grad_iters

    def __sigmoid(self, x):
        return 1.0 / (1.0 + np.e ** (-x))

    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

    def __add_intercept(self, X):
        return np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

    def fit(self, X, y):
        #print(X.dtype)
        X = self.__add_intercept(X)
        self.w = np.zeros(X.shape[1])

        for _ in range(self.gi):
            h = self.__sigmoid(np.dot(X, self.w))
            g = np.dot(X.T, (h - y)) / y.size
            self.w -=  (self.lr * g).astype(np.float64)
        pass

    def __predict_probability(self, X):
        X = self.__add_intercept(X)
        return self.__sigmoid(np.dot(X, self.w))

    def predict(self, X, threshold=0.5):
        return self.__predict_probability(X) >= threshold

KNN

In [487]:
class KNN():
    def __init__(self, neighbors=5):
        self.nn = neighbors
        self.breakraiser = 0
    
    def fit(self, X, y):
        self.X = X
        self.y = y.reshape((y.shape[0], 1))

    def __get_distances(self, p):
        t = (self.X - p)
        return np.sqrt((t**2).sum(1))

    def predict(self, X):
        n = X.shape[0]
        y_pred = np.zeros(n)
        y_sorted = np.zeros(self.nn)
        countF=0
        for i in range(n):
            d = self.__get_distances(X[i])
            for it,val in enumerate(np.argpartition(d, self.nn)[:self.nn]):
                y_sorted[it]=self.y[val]
            y_pred[i] = Counter(y_sorted).most_common(1)[0][0]
        return y_pred

Дерево решений

In [6]:
class Node():
  def __init__(self, predType):
    self.predType = predType
    self.iFeature = 0
    self.border = 0
    self.left = None
    self.right = None

class DecisionTree():
  def __init__(self, mDepth = 1, rf = False):
    self.mDepth = mDepth
    self.rf = rf

  def fit(self, X, y, maxFeatures = None):
    self.sizeY = len(set(y))
    self.setY  = set(y)
    self.dictY = {t:i for i,t in enumerate(self.setY)}
    #print(self.dictY)
    if not self.rf:
      Features = X.shape[1]
    else:
      ind = np.random.choice(X.shape[0], X.shape[0])
      X, y = X[tuple([ind])], y[tuple([ind])]
      if maxFeatures is None:
        Features = np.sqrt(X.shape[1]).astype(int)
      else:
        Features = maxFeatures
    self.features = np.sort(np.random.choice(X.shape[1], Features, replace = False))
    self.tree = self.UpdateTree(X, y)

  def predict(self, X):
    list = []
    for inputs in X:
      node = self.tree
      while node.left:
        if inputs[node.iFeature] < node.border:
          node = node.left
        else:
          node = node.right
      list.append(node.predType)
    return list

  def Split(self, X, y):
    m = y.size    
    if m <= 1:
      return None, None
    parent = [np.sum(y == c) for c in self.setY]
    bGini = 1.0 - sum((n / m) ** 2 for n in parent)
    bIdx, bThr = None, None
    dic=self.dictY
    for idx in self.features:
      borders, types = zip(*sorted(zip(X[:, idx], y)))
      left = [0] * self.sizeY
      right = parent.copy()
      for i in range(1, m):
        c = types[i - 1]
        #print(c)
        right[dic[c]] -= 1
        left[dic[c]] += 1
        giniLeft = 1.0 - sum((left[dic[x]] / i) ** 2 for x in self.setY)
        giniRight = 1.0 - sum((right[dic[x]] / (m - i)) ** 2 for x in self.setY)
        gini = (i * giniLeft + (m - i) * giniRight) / m
        if borders[i] == borders[i - 1]:
          continue
        if gini < bGini:
          bGini = gini
          bIdx = idx
          bThr = (borders[i] + borders[i - 1]) / 2
    return bIdx, bThr

  def UpdateTree(self, X, y, depth = 0):
    sPerClass = [np.sum(y == i) for i in self.setY]
    predType = np.argmax(sPerClass)
    node = Node(predType = predType)
    if depth < self.mDepth:      
      idx, thr = self.Split(X, y)
      if idx is not None:
        Lidx = X[:, idx] < thr
        lx, ly = X[Lidx], y[Lidx]
        rx, ry = X[~Lidx], y[~Lidx]
        node.iFeature = idx
        node.border = thr
        node.left = self.UpdateTree(lx, ly, depth + 1)
        node.right = self.UpdateTree(rx, ry, depth + 1)
    return node

Случайный лес

In [7]:
class RandomForest():
    def __init__(self, max_depth=5, n_estimators=100, max_features=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.forest = [None] * n_estimators

    def fit(self, X, y):
        for i in range(self.n_estimators):
            self.forest[i] = DecisionTree(
                self.max_depth, rf=True) 
            self.forest[i].fit(X, y)


    def predict(self, X):
        most_common = np.zeros(X.shape[0])
        preds = np.zeros((self.n_estimators, X.shape[0]))
        for i in range(self.n_estimators):
            preds[i] = self.forest[i].predict(X)
        for i in range(len(most_common)):
            most_common[i] = Counter(preds[:, i]).most_common(1)[0][0]
        return most_common.astype(int)

# **Первый датасет - Education**

Для этого датасета мы попробуем определить, наберёт ли поступающий более 65 баллов за математику при поступлении. 

In [3]:
Ed = pandas.read_csv("Education.csv")
required = list(Ed)
del required[required.index('math score')]
y = Ed["math score"]//65
x = pandas.get_dummies(Ed[required])
x.head()

,reading score,writing score,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none,gender_female,gender_male
0,72,74,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0
1,90,88,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,1,0
2,95,93,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0
3,57,44,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1
4,78,75,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1


## **Реализация на sklearn**

Логистическая регрессия

In [45]:
%%time
DisplayMetrics(LogisticRegression(), x, y)

precision: 0.8799537583332764
recall: 0.8762433321983882
train_accuracy: 0.8905000000000001
test_accuracy: 0.8789999999999999
Wall time: 921 ms


KNN

In [71]:
%%time
DisplayMetrics(KNeighborsClassifier(n_neighbors=5), x, y) 

precision: 0.8143771793285092
recall: 0.8132535190962157
train_accuracy: 0.87675
test_accuracy: 0.8160000000000001
Wall time: 1.21 s


Дерево решений

In [82]:
%%time
DisplayMetrics(DecisionTreeClassifier(max_depth=5), x, y)

precision: 0.8668050143905729
recall: 0.8643116558846897
train_accuracy: 0.8960000000000001
test_accuracy: 0.8669999999999998
Wall time: 221 ms


Случайный лес

In [83]:
%%time 
DisplayMetrics(RandomForestClassifier(n_estimators=50, max_depth=2), x, y) 

precision: 0.819732817881014
recall: 0.815822850429592
train_accuracy: 0.8275
test_accuracy: 0.8200000000000001
Wall time: 1.98 s


## **Реализация своими руками**

Логистическая регрессия

In [67]:
%%time
DisplayMetrics(LogReg(grad_iters=100), x, y)

precision: 0.27649999999999997
recall: 0.5
train_accuracy: 0.5529999999999999
test_accuracy: 0.5529999999999999
Wall time: 3.86 s


KNN

In [72]:
%%time
%prun KNN
DisplayMetrics(KNN(neighbors = 3), x, y)

precision: 0.7945751306345112
recall: 0.7921017953602224
train_accuracy: 0.8885
test_accuracy: 0.796
Wall time: 18.3 s


Дерево решений

In [78]:
%%time
DisplayMetrics(DecisionTree(mDepth = 4), x, y)

precision: 0.8618459805140043
recall: 0.8595848555399115
train_accuracy: 0.88225
test_accuracy: 0.8620000000000001
Wall time: 10 s


Случайный лес

In [79]:
%%time
DisplayMetrics(RandomForest(), x, y)

precision: 0.8032659792812895
recall: 0.7644576343452748
train_accuracy: 0.8190000000000002
test_accuracy: 0.7809999999999999
Wall time: 3min 53s


# **Второй датасет - netflix**

Для второго датасета попробуем определить сколько будет идти фильм, с точностью до десяти минут.

In [4]:
netflix = pandas.read_csv("netflix.csv")
required = list(netflix)
del required[required.index('duration')]
Y = netflix['duration']//10
X = pandas.get_dummies(netflix[required])
X.head()

,release_year,type_Movie,rating_G,rating_NC-17,rating_NR,rating_PG,rating_PG-13,rating_R,rating_TV-14,rating_TV-G,...,"listed_in_en_[9, 13, 16]","listed_in_en_[9, 13, 17]","listed_in_en_[9, 13, 27]","listed_in_en_[9, 15, 16]","listed_in_en_[9, 16, 17]","listed_in_en_[9, 16, 26]","listed_in_en_[9, 16, 29]","listed_in_en_[9, 16]","listed_in_en_[9, 17, 29]","listed_in_en_[9, 27]"
0,2019,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2014,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2017,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **Реализация на sklearn**

Логистическая регрессия

In [10]:
%%time
DisplayMetrics(LogisticRegression(), X, Y)

precision: 0.057289344471137336
recall: 0.06834272020190099
train_accuracy: 0.25750361821608114
test_accuracy: 0.24085745422122415
Wall time: 21.6 s


KNN

In [11]:
%%time
DisplayMetrics(KNeighborsClassifier(n_neighbors=5), X, Y) 

precision: 0.13253749959218564
recall: 0.1345102809396372
train_accuracy: 0.4635901776990036
test_accuracy: 0.22677487354765563
Wall time: 2min 35s


Дерево решений

In [12]:
%%time
DisplayMetrics(DecisionTreeClassifier(max_depth=15), X, Y) 

precision: 0.1756315357029777
recall: 0.14264320359351657
train_accuracy: 0.4476419473168492
test_accuracy: 0.28470710940122956
Wall time: 3.63 s


Случайный лес

In [13]:
%%time 
DisplayMetrics(RandomForestClassifier(n_estimators=50, max_depth=2), X, Y)

precision: 0.010579807488935174
recall: 0.048095238095238094
train_accuracy: 0.21998123098739578
test_accuracy: 0.21998112158688748
Wall time: 1.89 s


## **Реализация своими руками**

Логистическая регрессия

In [14]:
%%time
DisplayMetrics(LogReg(), X, Y)

precision: 0.00011279478355085573
recall: 0.048095238095238094
train_accuracy: 0.002345215792091169
test_accuracy: 0.0023452162761642146
Wall time: 1min 16s


KNN

In [489]:
%%time
DisplayMetrics(KNN(neighbors = 8), X, Y)

precision: 0.22260365975258342
recall: 0.23779948153162822
train_accuracy: 0.43058106608404023
test_accuracy: 0.23779948153162822
Wall time: 1h 14min 46s


Дерево решений

In [490]:
%%time
DisplayMetrics(DecisionTree(), X, Y)

precision: 0.06381055279650785
recall: 0.2258493909923
train_accuracy: 0.2261362023355237
test_accuracy: 0.2258493909923
Wall time: 2h 9min 14s


Случайный лес

In [84]:
%%time
DisplayMetrics(RandomForest(max_depth=2,n_estimators=2), X, Y)

precision: 0.01929252406453833
recall: 0.048888888888888885
train_accuracy: 0.203446473257508
test_accuracy: 0.20309952721408564
Wall time: 5min 45s


## Вывод ##

Реализация своими руками сильно уступает в скорости реализации на sklearn. Причиной этому может быть как неоптимизированность моего кода(например привидения типа X к float32 дало прибавку в скорости в 8 раз), так и хорошая оптимизация кода sklearn. За исключением времени, все показатели оказалась на уровне с sklearn во всех случаях, кроме логистической регресии, вероятно я сильно накосячил в её реализации. При этом в первом датасете они оказалась значительно выше, чем во втором, как впрочем и должно быть, т.к. в первом датасете есть данные с оценками за другие предметы, и очевидно, что корреляция между ними и оценкой за математику довольно высока, во втором же датасете надо определить время по режесёрам и рейтингу и году выхода, что явно имеет довольно низкую корреляцию.

Логистическая регрессия показала неплохую точность в 1-ом датасете, но имеет наихудшее показатели во 2-ом.
KNN имеет неплохие результаты в 1-ом и 2-ом датасете, однако для второго датасета выполнялся слишком долго по сравнению с другими моделями.
Во всех случаях дерево решений показало довольно неплохие показатели и выполнялось довольно быстро. 
Случайный лес имеет примерно одинаковое время выполнения для первого и второго датасета, вкупе  с неплохой точностью, однако уступает дереву решений в обоих датасетах. 

Пожалуй главным выводом будет никогда не пользоваться реализацией своими руками, т.к. она очень медленная(и при этом на втором датасете реализация KNN оказалась в 2 раза быстрее чем Дерево решений, что вызывает много вопросов).